# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
orders=pd.read_csv("Orders.csv")
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [3]:
"""The first group is VIP Customers whose aggregated expenses at your global chain stores are above the 95th percentile (aka. 0.95 quantile).
    
The second group is Preferred Customers whose aggregated expenses are between the 75th and 95th percentile."""

sub_prob1=pd.DataFrame(orders.groupby(by=["CustomerID"])["amount_spent"].sum())
sub_prob1.sort_values(by="amount_spent",ascending=False, inplace=True)
sub_prob1.columns=["sum_amount_spent"]
sub_prob1.head()

,sum_amount_spent
CustomerID,
14646,280206.02
18102,259657.30
17450,194550.79
16446,168472.50
14911,143825.06


In [4]:
sub_prob2_95=sub_prob1.quantile(q=.95)
sub_prob2_75=sub_prob1.quantile(q=.75)
print(sub_prob2_95)
print(sub_prob2_75)

sub_prob1[sub_prob1["sum_amount_spent"]>sub_prob2_95.item()]
sub_prob1[sub_prob1["sum_amount_spent"]>sub_prob2_75.item()]

sum_amount_spent    5840.182
Name: 0.95, dtype: float64
sum_amount_spent    1661.64
Name: 0.75, dtype: float64


,sum_amount_spent
CustomerID,
14646,280206.02
18102,259657.30
17450,194550.79
16446,168472.50
14911,143825.06
...,...
17656,1674.69
16115,1667.97
12912,1662.30


In [5]:
def status(x):
    if x>=sub_prob2_95.item():
        return 'VIP'
    elif (x<sub_prob2_95.item()) & (x>=sub_prob2_75.item()):
                return 'Preferred'

sub_prob1["status"]=sub_prob1["sum_amount_spent"].apply(status)

orders2=orders.merge(sub_prob1, how="inner", left_on="CustomerID", right_on="CustomerID")
orders2.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent,sum_amount_spent,status
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,5391.21,Preferred
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,5391.21,Preferred
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,5391.21,Preferred
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,5391.21,Preferred
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,5391.21,Preferred


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [6]:
check=pd.DataFrame(orders2[orders2["status"]=="VIP"].groupby(by="CustomerID")["Country"].first())
#orders2.groupby(by=["status","CustomerID"])["Country"].first()

print(check["Country"].value_counts().index[0], "with", check["Country"].value_counts()[0], "VIP Customers")

United Kingdom with 177 VIP Customers


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [15]:
check2=pd.DataFrame(orders2[(orders2["status"]=="VIP")|(orders2["status"]=="Preferred")].groupby(by="CustomerID")["Country"].first())
check2["Country"].value_counts()
print(check2["Country"].value_counts().index[0], "with", check2["Country"].value_counts()[0], "VIP+Preferred Customers combined")

United Kingdom with 932 VIP+Preferred Customers combined
